In [1]:
import os
import numpy as np
import pandas as pd
import math
import csv

import transformers
from tqdm.notebook import trange, tqdm

from sentence_transformers import SentenceTransformer, SentencesDataset, losses
from sentence_transformers.readers import STSDataReader, TripletReader
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, BinaryEmbeddingSimilarityEvaluator, SequentialEvaluator, TripletEvaluator
from sentence_transformers.readers.InputExample import InputExample

import torch
from torch.utils.data import DataLoader, RandomSampler

from scipy.spatial.distance import cdist


In [2]:
# evaluate final mdoel on train data
"""
this_model_path = 'msmarco/models/test_model4/model4'
this_model =  SentenceTransformer('roberta-large-nli-stsb-mean-tokens') #SentenceTransformer(this_model_path)
this_train_dataloader = torch.load(os.path.join('msmarco/models/test_model4', 'train_dataloader.pth'))

evaluator1 = BinaryEmbeddingSimilarityEvaluator(this_train_dataloader)
evaluator2 = EmbeddingSimilarityEvaluator(this_train_dataloader)
this_evaluator = SequentialEvaluator([evaluator1, evaluator2])

this_model.evaluate(this_evaluator, output_path=os.path.join(this_model_path, 'train_set_performance_beforetrain'))
"""

"\nthis_model_path = 'msmarco/models/test_model4/model4'\nthis_model =  SentenceTransformer('roberta-large-nli-stsb-mean-tokens') #SentenceTransformer(this_model_path)\nthis_train_dataloader = torch.load(os.path.join('msmarco/models/test_model4', 'train_dataloader.pth'))\n\nevaluator1 = BinaryEmbeddingSimilarityEvaluator(this_train_dataloader)\nevaluator2 = EmbeddingSimilarityEvaluator(this_train_dataloader)\nthis_evaluator = SequentialEvaluator([evaluator1, evaluator2])\n\nthis_model.evaluate(this_evaluator, output_path=os.path.join(this_model_path, 'train_set_performance_beforetrain'))\n"

In [3]:
torch.cuda.empty_cache()

In [4]:
my_model_path = '/run/media/root/Windows/Users/agnes/Downloads/data/msmarco/train_results/test_wiki'

In [5]:
# load model
model_wiki = SentenceTransformer('bert-base-wikipedia-sections-mean-tokens')

In [6]:
dev_dataloader = torch.load(os.path.join(my_model_path, 'dev_dataloader.pth'))

In [7]:
train_dataloader =  torch.load(os.path.join(my_model_path, 'train_dataloader.pth'))

In [8]:
evaluator1 = BinaryEmbeddingSimilarityEvaluator(dev_dataloader)
evaluator2 = EmbeddingSimilarityEvaluator(dev_dataloader)
evaluator3 = TripletEvaluator(dev_dataloader)
evaluator = SequentialEvaluator([evaluator3])

In [9]:
#model_1.evaluate(evaluator, output_path=os.path.join(my_model_path, 'dev_set_performance_pretrain'))

In [10]:
optimizer_class = transformers.AdamW
optimizer_params = {'lr': 2e-4, 'eps': 1e-6, 'correct_bias': False}
train_loss = losses.TripletLoss(model=model_wiki)

In [11]:
num_epochs = 20000
warmup_steps = math.ceil(len(train_dataloader.dataset)*num_epochs / train_dataloader.batch_size*0.05) #5% of train data for warm-up

model_wiki.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          steps_per_epoch=8000,
          warmup_steps=warmup_steps,
          optimizer_class=optimizer_class,
          optimizer_params=optimizer_params,
          output_path=os.path.join(my_model_path, 'model_2')) # works only when you have an evaluator

Iteration:   1%|          | 75/8000 [06:24<11:07:14,  5.05s/it]

KeyboardInterrupt: 

In [ ]:
#model_1.evaluate(evaluator)

In [ ]:
model_1.save(os.path.join(my_model_path, 'model2_final'))

In [5]:
sentences = ["aap noot mies", "hi how are you"]

embeddings = model_1.encode(sentences)
print(embeddings[0].shape)
sims = cdist(embeddings[0].reshape(-1,1).T, embeddings[1:], "cosine")[0]
sims

(1024,)


array([0.90372015])